# Mask R-CNN for Keypoint Detection

Example showing how to do keypoint detection with Mask R-CNN

In [1]:
%matplotlib inline
import importlib
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load COCO Datasets for Keypoint Detection

In [2]:
# Import COCO classes
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))

import coco_keypoints
importlib.reload(coco_keypoints)

# COCO dataset dir
COCO_DATA_DIR = "A:/Programming/DeepLearningDatasets/coco" if os.path.isdir("A:/Programming/DeepLearningDatasets/coco") else os.path.join(ROOT_DIR, "data/coco")

# Prepare dataset
dataset_train = coco_keypoints.CocoDataset()
dataset_train.load_coco(COCO_DATA_DIR, subset="train", year="2017", auto_download=True)
dataset_train.prepare()

dataset_val = coco_keypoints.CocoDataset()
dataset_val.load_coco(COCO_DATA_DIR, subset="val", year="2017", auto_download=True)
dataset_val.prepare()

Will use images in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017
Will use annotations in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/annotations/person_keypoints_train2017.json
loading annotations into memory...
Done (t=8.45s)
creating index...
index created!
Will use images in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/val2017
Will use annotations in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/annotations/person_keypoints_val2017.json
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


### Setup Training

In [3]:
class TrainConfig(Config):
    NAME = "coco_keypoints"
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1 # IMAGES_PER_GPU = 2
    GPU_COUNT = 1
    
    # We use resnet50
    BACKBONE = "resnet50"
    
    # Set number of classes
    NUM_CLASSES = dataset_train.num_classes
    NUM_KEYPOINTS = dataset_train.num_kp_classes
    
    # DEBUG:
    STEPS_PER_EPOCH = 1000
    
    USE_MINI_MASK = False
    TRAIN_ROIS_PER_IMAGE = 100
    #MAX_GT_INSTANCES = 2
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

config = TrainConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MIN

In [4]:
importlib.reload(modellib)

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True,
                   #exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_mask"])

Shapes
target_mask.shape (1, ?, ?, ?, ?)
1
target_masks.shape (1, ?, ?, ?, ?)
target_kp_ids.shape (1, ?, 17)
pred_masks.shape (?, 100, 28, 28, 17)
2
target_masks.shape (?, ?, ?)
pred_masks.shape (?, ?, ?)
y_true.shape (?, ?, ?)
y_pred.shape (?, ?, ?)
loss Tensor("mrcnn_mask_loss/cond/Merge:0", dtype=float32)
1
target_masks.shape (?, 100, 28, 28, 17)
target_kp_ids.shape (?, 17)
pred_masks.shape (?, 100, 28, 28, 17)
2
target_masks.shape (?, ?, ?)
pred_masks.shape (?, ?, ?)
y_true.shape (?, ?, ?)
y_pred.shape (?, ?, ?)
loss Tensor("mrcnn_mask_loss/cond_1/Merge:0", dtype=float32)


## Train

In [ ]:
# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers="heads")

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /datasets/home/78/378/cs252cas/Pose_RCNN/logs/coco_keypoints20180528T2003/mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistri

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2023: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'
ERROR:root:Error processing image {'id': 550303, 'source': 'coco', 'path': '/datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017/000000550303.jpg', 'width': 640, 'height': 427, 'annotations': [{'segmentation': [[92.12, 219.44, 90.2, 202.17, 97.87, 197.37, 108.43, 200.25, 108.43, 217.52, 108.43, 223.28, 127.62, 236.72, 127.62, 252.07, 117.07, 256.87, 117.07, 276.06, 116.11, 298.13, 110.35, 365.3, 116.11, 370.09

Epoch 1/1


ERROR:root:Error processing image {'id': 528929, 'source': 'coco', 'path': '/datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017/000000528929.jpg', 'width': 640, 'height': 480, 'annotations': [{'segmentation': [[414.2, 162.6, 422.53, 185.21, 428.08, 184.81, 437.6, 186.4, 447.12, 186.4, 453.07, 186.4, 457.04, 185.61, 460.61, 181.64, 460.21, 176.09, 458.23, 171.33, 455.45, 165.38, 453.47, 161.41, 449.5, 158.24, 446.33, 155.86, 447.12, 152.68, 448.71, 147.13, 449.1, 139.99, 449.1, 138.8, 447.12, 132.06, 438, 126.11, 428.87, 126.11, 422.53, 130.07, 421.34, 133.25, 420.94, 138.8, 420.94, 144.75, 421.73, 149.51, 421.73, 153.08, 421.73, 155.86, 422.53, 159.43, 422.53, 160.22, 420.54, 162.6, 418.56, 163.79]], 'num_keypoints': 8, 'area': 1863.31855, 'iscrowd': 0, 'keypoints': [422, 147, 2, 427, 143, 2, 422, 143, 2, 441, 146, 2, 0, 0, 0, 452, 168, 2, 418, 167, 2, 435, 184, 2, 0, 0, 0, 410, 172, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'image_id': 528929, 'bbox':

   5/1000 [..............................] - ETA: 42:21 - loss: 3.9234 - rpn_class_loss: 0.0767 - rpn_bbox_loss: 2.5604 - mrcnn_class_loss: 0.3270 - mrcnn_bbox_loss: 0.9029 - mrcnn_mask_loss: 0.0564

...

 931/1000 [==========================>...] - ETA: 39s - loss: 1.6299 - rpn_class_loss: 0.0478 - rpn_bbox_loss: 0.8364 - mrcnn_class_loss: 0.1693 - mrcnn_bbox_loss: 0.4531 - mrcnn_mask_loss: 0.1233

ERROR:root:Error processing image {'id': 206102, 'source': 'coco', 'path': '/datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017/000000206102.jpg', 'width': 500, 'height': 333, 'annotations': [{'segmentation': [[262.07, 280.82, 260.58, 253.2, 259.08, 239.76, 253.11, 236.77, 250.87, 223.33, 230.71, 214.37, 226.98, 197.2, 225.48, 158.37, 237.43, 146.43, 241.16, 138.21, 244.15, 126.27, 255.35, 124.03, 262.07, 132.99, 263.56, 141.95, 264.31, 147.17, 271.03, 150.91, 273.27, 177.04, 277, 191.23, 259.08, 193.47, 245.64, 197.95, 259.83, 200.93, 273.27, 198.69, 279.99, 197.95, 283.72, 217.36, 291.19, 241.25, 312.09, 262.9, 304.63, 267.38, 287.46, 256.93, 283.72, 244.98, 275.51, 233.78, 271.03, 251.7, 268.79, 262.16, 273.27, 276.34, 268.79, 279.33, 264.31, 278.58]], 'num_keypoints': 16, 'area': 5203.9133, 'iscrowd': 0, 'keypoints': [257, 147, 2, 258, 143, 2, 254, 144, 2, 0, 0, 0, 246, 144, 2, 262, 155, 2, 234, 163, 2, 267, 175, 2, 233, 193, 2, 269, 182, 2, 254, 191, 2, 259, 201, 2, 241, 

 932/1000 [==========================>...] - ETA: 39s - loss: 1.6290 - rpn_class_loss: 0.0478 - rpn_bbox_loss: 0.8358 - mrcnn_class_loss: 0.1692 - mrcnn_bbox_loss: 0.4529 - mrcnn_mask_loss: 0.1233

ERROR:root:Error processing image {'id': 20629, 'source': 'coco', 'path': '/datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017/000000020629.jpg', 'width': 640, 'height': 476, 'annotations': [{'segmentation': [[400.65, 108.09, 406.93, 95.52, 418.04, 91.18, 431.57, 86.83, 438.82, 75.23, 447.03, 67.5, 458.98, 76.92, 464.78, 78.37, 466.23, 84.17, 462.85, 92.38, 457.05, 95.77, 471.54, 113.16, 470.58, 125.72, 461.88, 124.27, 460.91, 124.76, 458.5, 156.16, 435.31, 111.71], [469.13, 133.94, 470.58, 184.67, 481.21, 189.5, 482.17, 197.72, 473.96, 194.33, 470.58, 189.5, 465.75, 174.04, 465.26, 133.94]], 'num_keypoints': 11, 'area': 2709.1656, 'iscrowd': 0, 'keypoints': [464, 83, 2, 0, 0, 0, 460, 78, 2, 0, 0, 0, 446, 79, 2, 464, 109, 2, 415, 99, 2, 463, 158, 1, 406, 147, 1, 469, 187, 1, 428, 135, 1, 445, 184, 1, 416, 184, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'image_id': 20629, 'bbox': [400.65, 67.5, 81.52, 130.22], 'category_id': 1, 'id': 187832}, {'segmentation': [[638.74, 421.47, 612

 944/1000 [===========================>..] - ETA: 32s - loss: 1.6271 - rpn_class_loss: 0.0482 - rpn_bbox_loss: 0.8329 - mrcnn_class_loss: 0.1697 - mrcnn_bbox_loss: 0.4532 - mrcnn_mask_loss: 0.1233

In [ ]:
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers="4+",
            augmentation=augmentation)

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers="all",
            augmentation=augmentation)

Explanation of losses:

- rpn_class_loss: "How well does the network separate positive from negative anchors?"
- rpn_bbox_loss: "How accurate are proposed the bounding boxes?"
- mrcnn_class_loss: "How well does the network distinguish people from background RoIs?" (really, this is the same as 1.)
- mrcnn_bbox_loss: "Bounding box refinement loss" (is this the same as 2.?)
- mrcnn_mask_loss: "How accurate are the predicted keypoints?" (per mask softmax cross-entropy loss)

In [ ]:
print("Done")

In [ ]:
class InferenceConfig(coco_keypoints.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    BACKBONE = "resnet50"
    NUM_CLASSES = dataset_train.num_classes
    NUM_KEYPOINTS = dataset_train.num_kp_classes
    USE_MINI_MASK = False
    #MAX_GT_INSTANCES = 2
    
inference_config = InferenceConfig()
inference_config.display()

In [ ]:
importlib.reload(modellib)

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Inspect Model

In [ ]:
importlib.reload(modellib)
importlib.reload(visualize)

# Load a test image
image_id = dataset_val.image_ids[10]

# Get ground truth masks and bboxes
image, image_meta, gt_class_ids, gt_bbox, gt_kp_masks, gt_kp_ids = \
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("image", image)
log("gt_class_ids", gt_class_ids)
log("gt_kp_ids", gt_kp_ids)
log("gt_bbox", gt_bbox)
log("gt_masks", gt_kp_masks)

visualize.display_keypoints(image, gt_bbox, gt_kp_masks, gt_kp_ids,
                            figsize=(8, 8), skeleton=dataset_train.skeleton)

In [ ]:
def verify_masks(masks):
    if len(masks.shape) == 3: masks = np.expand_dims(masks, axis=0)
    for i in range(masks.shape[0]):
        for j in range(masks.shape[-1]):
            if np.sum(masks[i, :, :, j]) > 1:
                print("Mask [%i, :, :, %i] sum = %i" % (i, j, np.sum(masks[i, :, :, j])))
verify_masks(gt_kp_masks)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

results = model.detect([image], verbose=1)
r = results[0]

In [ ]:
masks = r["masks"]
print(masks.shape)
print(masks.dtype)

#verify_masks(masks)

bbox, kp_masks, kp_ids = r["rois"], r["masks"], np.repeat(np.expand_dims(np.arange(17), axis=0), 12, axis=0)

log("image", image)
log("class_ids", r['class_ids'])
log("kp_ids", kp_ids)
log("bbox", bbox)
log("masks", masks)

%matplotlib inline
import matplotlib.pyplot as plt

_, axes = plt.subplots(4, 5, figsize=(15, 15))
for i, ax in enumerate(axes.flatten()):
    ax.axis("off")
    if i >= 17: continue
    #x, y = i % 4, i // 4
    
    ax.set_title(dataset_train.kp_class_names[i+1])
    ax.imshow(masks[0, :, :, i])


plt.show()

visualize.display_keypoints(image, bbox[0:1], kp_masks[0:1],
                            figsize=(8, 8), skeleton=dataset_train.skeleton)

In [ ]:
print(r['class_ids'])
print(r["scores"])

In [ ]:
#visualize.display_keypoints(original_image, r['rois'], r['masks'], r['class_ids'],
#                            dataset_val.class_names, skeleton=None)

In [ ]:
# Get predictions of mask head
mrcnn = model.run_graph([original_image], [
    ("detections", model.keras_model.get_layer("mrcnn_detection").output),
    ("masks", model.keras_model.get_layer("mrcnn_mask").output),
])

# Get detection class IDs. Trim zero padding.
det_class_ids = mrcnn['detections'][0, :, 4].astype(np.int32)
#print(det_class_ids)
#det_count = np.where(det_class_ids == 0)[0][0]
#det_class_ids = det_class_ids[:det_count]

#print("{} detections: {}".format(
#    det_count, np.array(dataset.class_names)[det_class_ids]))

In [ ]:
# Masks
det_boxes = utils.denorm_boxes(mrcnn["detections"][0, :, :4], image.shape[:2])
det_mask_specific = np.array([mrcnn["masks"][0, i, :, :, c] 
                              for i, c in enumerate(det_class_ids)])
det_masks = np.array([utils.unmold_mask(m, det_boxes[i], image.shape)
                      for i, m in enumerate(det_mask_specific)])
log("det_mask_specific", det_mask_specific)
log("det_masks", det_masks)

In [ ]:
visualize.display_images(det_mask_specific[:4] * 255, cmap="Blues", interpolation="none")